In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder\
    .master("local")\
    .appName("lesson_15")\
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/05 15:48:54 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
employees = sc.parallelize([[1, 'Anton'], [2, 'Vasiliy'], [3, 'Anna']]).toDF(("id", "name"))
salary = sc.parallelize([[1, 2000], [2, 3000], [3, 4000]]).toDF(("id","salary"))

employees.show()

In [4]:
employees.show()
salary.show()

+---+-------+
| id|   name|
+---+-------+
|  1|  Anton|
|  2|Vasiliy|
|  3|   Anna|
+---+-------+

+---+------+
| id|salary|
+---+------+
|  1|  2000|
|  2|  3000|
|  3|  4000|
+---+------+



In [5]:
employees.join(salary, 'id').show()

+---+-------+------+
| id|   name|salary|
+---+-------+------+
|  1|  Anton|  2000|
|  3|   Anna|  4000|
|  2|Vasiliy|  3000|
+---+-------+------+



In [6]:
employees.join(salary, 'id').explain()  # query plan - shuffling (join tables from different nodes (partitions))

== Physical Plan ==
*(5) Project [id#0L, name#1, salary#5L]
+- *(5) SortMergeJoin [id#0L], [id#4L], Inner
   :- *(2) Sort [id#0L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(id#0L, 200), ENSURE_REQUIREMENTS, [id=#105]
   :     +- *(1) Filter isnotnull(id#0L)
   :        +- *(1) Scan ExistingRDD[id#0L,name#1]
   +- *(4) Sort [id#4L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#4L, 200), ENSURE_REQUIREMENTS, [id=#111]
         +- *(3) Filter isnotnull(id#4L)
            +- *(3) Scan ExistingRDD[id#4L,salary#5L]




In [7]:
employees.join(F.broadcast(salary), 'id').explain() # broadcast salary - is copy to each nodes (partitions)

== Physical Plan ==
*(2) Project [id#0L, name#1, salary#5L]
+- *(2) BroadcastHashJoin [id#0L], [id#4L], Inner, BuildRight, false
   :- *(2) Filter isnotnull(id#0L)
   :  +- *(2) Scan ExistingRDD[id#0L,name#1]
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [id=#146]
      +- *(1) Filter isnotnull(id#4L)
         +- *(1) Scan ExistingRDD[id#4L,salary#5L]




----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 60254)
Traceback (most recent call last):
  File "/usr/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.8/socketserver.py", line 720, in __init__
    self.handle()
  File "/home/user/.local/lib/python3.8/site-packages/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/home/user/.local/lib/python3.8/site-packages/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/home/user/.local/lib/python3.8/site-packages/pyspark/accumulators.py", line 239, in accum_updates
    num_updates = read_int(self.r